This is the example based on my codebar coaching.

We can use an agent to return back the next step as we saw in the JOKE example.

We have a list of reports and their uses and a user can ask a question with the agent returning back the most useful report to be run.

This can them be processes with more application logic.

This is ROUTING or IF/ELSE type decision making.

As we saw in 03 FAQ, we could use this to create a filter for the next agent so that it can be more selective about data retrieval.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

c:\Users\mrcra\Desktop\OK-django-con-eu\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# For variation use a function to get the LLM client based on the user's choice


def get_llm_client(llm_choice):

    if llm_choice == "GROQ":

        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )

        return client

    elif llm_choice == "OPENAI":

        load_dotenv()  # load environment variables from .env fil

        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

        return client

    else:

        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-1WUVgv...
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


In [4]:
# A request to the LLM is stateless so we will always need to pass all the data that is needed each time.

# `history` is just that - a record of what has gone on before so that the LLM can have context to answer the query.


# We will use GRADIO as our UI.
def chat(message, history):
    # history is part of the gradio ChatInterface and it stores previous answers
    messages = (
        [{"role": "system", "content": system_message}]
        # + history ## groq gives error as it adds metadata
        + [{"role": "user", "content": message}]
    )
    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)
    # ====================
    # AI bit
    stream = client.chat.completions.create(
        model=MODEL, messages=messages, stream=True, temperature=0.0
    )

    # Just UI implementation
    response = ""
    for stream_so_far in stream:
        response += stream_so_far.choices[0].delta.content or ""
        yield response

In [5]:
system_message = """
You are a report selection agent. 
You are very good at returning the best report to answer a user's question. 
For example, if a user wants a joke, you reply with |TOOL: **get_joke**|
Another example, if they want a **TOTAL SALES** report then |TOOL: **get_sales**| would be the best report.
We use this format so we can extract out reports from your answer.
"""

In [6]:
# ** just for bold in front end but does add some importance too.

REPORTS = [
    "For information on Sprints use this tool => **get_sprint**.",
    "For information on Django Conference Venue and loation use this tool => **get_conf_info**",
    "For information on Grants and Support use this tool => **get_grant_info**",
    "For information on Talks and workshops use this tool => **get_talk_info**",
    "For information on Sprints use this tool => **get_sprint**.",
    "For weather use this tool => **get_weather**.",
    "For hotel booking use this tool => **get_hotel_booking**.",
    "For car hire use this tool => **get_car_hire**.",
    "For flight booking use this tool => **get_flight**.",
    "For total sales use this tool => **get_sales**.",
    "For site statistics use this tool => **get_site_statistics**.",
    "For jokes use this tool => **get_joke**.",
    "To add two numbers use this tool => **get_adder**.",
    "To compute loss function => **get_loss**.",
    "To critique an article prior to publication but not after publication use this tool => **get_article_review**.",
    "For all other requests use this tool => **customer_service_agent**.",
]

# Again, 'character and instruction' along with knowledge
system_message += "\n" + "\n".join(REPORTS)

If we combine this ROUTER as a precursor to the FAQ pattern, we one agent selecting the domain, and another agent answering the question.

This is a MULTI-AGENT system - we can have a range of design patterns that can be used to create a MULTI-AGENT system.

<img src="./images/antrhopic-workflows.png" width=700px>

https://www.anthropic.com/engineering/building-effective-agents

In [7]:
# We use Gradio for a chat interface
# prompt: I am want a plane to Rome then an auto to Paris

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "\nYou are a report selection agent. \nYou are very good at returning the best report to answer a user's question. \nFor example, if a user wants a joke, you reply with |TOOL: **get_joke**|\nAnother example, if they want a **TOTAL SALES** report then |TOOL: **get_sales**| would be the best report.\nWe use this format so we can extract out reports from your answer.\n\nFor information on Sprints use this tool => **get_sprint**.\nFor information on Django Conference Venue and loation use this tool => **get_conf_info**\nFor information on Grants and Support use this tool => **get_grant_info**\nFor information on Talks and workshops use this tool => **get_talk_info**\nFor information on Sprints use this tool => **get_sprint**.\nFor weather use this tool => **get_weather**.\nFor hotel booking use this tool => **get_hotel_booking**.\nFor car hire use this tool => **get_car_hire**.\nFor flight booking use this tool => **get_flight*